In [27]:
''' 
Link to Documentation: https://github.com/ultralytics/yolov5

RUN IN TERMINAL BEFORE PROCEEDED IN THE 'fashion-forecast' DIRECTORY:
>   pip install ultralytics
>   git clone https://github.com/ultralytics/yolov5  
>   cd yolov5
>   pip install -r requirements.txt  
>   cd ..
'''

" \nLink to Documentation: https://github.com/ultralytics/yolov5\n\nRUN IN TERMINAL BEFORE PROCEEDED IN THE 'fashion-forecast' DIRECTORY:\n>   pip install ultralytics\n>   git clone https://github.com/ultralytics/yolov5  \n>   cd yolov5\n>   pip install -r requirements.txt  \n>   cd ..\n"

In [1]:
import torch
import numpy as np
import pandas as pd
import os
from PIL import Image

%matplotlib inline 
from matplotlib import pyplot as plt

In [2]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5n', pretrained=True) #force_reload=True

Using cache found in C:\Users\cardon/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-2-16 Python-3.8.5 torch-2.0.1+cpu CPU

Fusing layers... 
YOLOv5n summary: 213 layers, 1867405 parameters, 0 gradients
Adding AutoShape... 


In [3]:
'''
CODE TO CHANGE DEVICE:
model.cpu()  # CPU
model.cuda()  # GPU
model.to(device)  # i.e. device=torch.device(0)
'''

'\nCODE TO CHANGE DEVICE:\nmodel.cpu()  # CPU\nmodel.cuda()  # GPU\nmodel.to(device)  # i.e. device=torch.device(0)\n'

In [10]:
img_list = []
img_names = []
directory = '../unpack-data/'
#trial_size = 10
seasons_only_csv = pd.read_csv('filtered_style_stats.csv')
counter = 0
for idx, img_name in enumerate(os.listdir(directory)):
    if seasons_only_csv['new_pic_name'].iloc[idx] != img_name:
        continue
    #if counter == trial_size:
    #    break
    img_path = os.path.join(directory, img_name)
    pil_img = Image.open(img_path)
    np_img = np.array(pil_img)
    #plt.imshow(np_img)             # used to display the images
    #plt.show()
    img_list.append(np_img)
    img_names.append(str(img_path))
    counter += 1

000ecstaSY000-75818.jpg 000ecstaSY000-75818.jpg False
000ecstaSY000-75819.jpg 000ecstaSY000-75819.jpg False
000ecstaSY000-75820.jpg 000ecstaSY000-75820.jpg False
0011zm-129448.jpg 0011zm-129448.jpg False
0011zm-129449.jpg 0011zm-129449.jpg False
0011zm-129450.jpg 0011zm-129450.jpg False
0011zm-129451.jpg 0011zm-129451.jpg False
0011zm-129452.jpg 0011zm-129452.jpg False
0011zm-129453.jpg 0011zm-129453.jpg False
0011zm-129454.jpg 0011zm-129454.jpg False
0011zm-129455.jpg 0011zm-129455.jpg False


In [6]:
results = model(img_list)  # inference
results.save()   # cropped detections dictionary
#results.show()

Saved 10 images to runs\detect\exp


In [7]:
detection_df = results.pandas().xyxy
print(len(detection_df))
'''
for idx, df in enumerate(detection_df):
    person_df = df[df['name']=='person']
    print(person_df.index)
'''

10


"\nfor idx, df in enumerate(detection_df):\n    person_df = df[df['name']=='person']\n    print(person_df.index)\n"

In [8]:
"""Takes the current results and saves only the information from the maximum 'person' detections"""

cleaned_df_lst = []
for idx, df in enumerate(detection_df):
    person_df = df[df['name']=='person']
    if person_df.empty == True:
        continue
    if (len(person_df.index) == 1):
        row = person_df.iloc[0].tolist()
        row.append(idx)
        cleaned_df_lst.append(row)
        continue
    # multiple detections -> choose the highest confidence of person
    maxValueIndex = person_df['confidence'].idxmax()
    row = person_df.iloc[maxValueIndex].tolist()
    row.append(idx)
    cleaned_df_lst.append(row)

col_names = list(detection_df[0].columns) + ['df_idx']
cleaned_df = pd.DataFrame(cleaned_df_lst, columns = col_names)
print(len(cleaned_df.index))
#print(cleaned_df)


9


In [9]:
"""Takes the cleaned df data and crops each image to it's detected bounding box dimensions"""
directory = 'C:/Users/cardon/OneDrive - Rensselaer Polytechnic Institute/SEM8/RCOS/unpack-data'
save_dir = 'C:/Users/cardon/OneDrive - Rensselaer Polytechnic Institute/SEM8/RCOS/crop-data'
df_idx = 0
for idx, img_name in enumerate(os.listdir(directory)):
    if idx == 8:
        break
    if idx not in cleaned_df['df_idx'].values:
        continue

    xmin, ymin, xmax, ymax = cleaned_df.loc[df_idx][['xmin', 'ymin', 'xmax', 'ymax']]
    #print(int(xmin), int(ymin), int(xmax), int(ymax))
    df_idx += 1

    img_path = os.path.join(directory, img_name)
    #print(img_path)
    pil_img = Image.open(img_path)
    np_img = np.array(pil_img)
    #plt.imshow(np_img)             
    #plt.show()
    crop_img = np_img[int(ymin):int(ymax), int(xmin):int(xmax)]
    #plt.imshow(crop_img)             
    #plt.show()
    save_path = os.path.join(save_dir, (img_name+'.jpg'))
    save_im = Image.fromarray(crop_img)
    save_im.save(save_path)